In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.svm import SVR, SVC

#torch.manual_seed(100)
np.random.seed(100)

In [2]:
# load prepprocessed data
tool_length = 50
npzfile = np.load(f'../preprocessed/neutouch_tool_{tool_length}.npz')

# scale a bit
X = npzfile['signals'] / 1000
y = npzfile['labels'] * 100

print(X.shape)

# fft
X = np.abs(np.fft.fft(X)) / 10
X = np.reshape(X, (X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

(964, 80, 61)


In [3]:
estimator = SVR(kernel='rbf', max_iter=5000)
param_grid = { 'C': [1, 3, 10, 30, 100] }
gs_estimator = GridSearchCV(estimator=estimator, param_grid=param_grid,
                            scoring='neg_mean_absolute_error',
                            cv=4, n_jobs=-1, refit=True)
gs_estimator.fit(X_train, y_train)
test_loss = -gs_estimator.score(X_test, y_test)
print(test_loss)

2.725273412720616


In [4]:
import pickle
pickle.dump(gs_estimator, open(f'tool{tool_length}_fft_svmrbf.pkl', 'wb'))

In [5]:
gs_estimator.best_estimator_

SVR(C=30, max_iter=5000)